<a href="https://colab.research.google.com/github/Trickshotblaster/nn-practices/blob/main/TrigramLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import math

In [2]:
import requests

url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
url = 'https://raw.githubusercontent.com/karpathy/makemore/master/names.txt'
url = 'https://raw.githubusercontent.com/Trickshotblaster/nn-practices/main/TrigramLM.ipynb'
response = requests.get(url)

with open('file.txt', 'wb') as f:
  f.write(response.content)

In [3]:
with open('file.txt', 'r') as data:
  dataset = data.readlines()

with open('file.txt', 'r') as data:
  raw_data = data.read()


In [4]:
raw_data[:10]

'{\n  "nbfor'

In [5]:
dataset = raw_data.split("\n")

In [6]:
dataset[:10]

['{',
 '  "nbformat": 4,',
 '  "nbformat_minor": 0,',
 '  "metadata": {',
 '    "colab": {',
 '      "provenance": [],',
 '      "include_colab_link": true',
 '    },',
 '    "kernelspec": {',
 '      "name": "python3",']

In [7]:
train_set = dataset[:math.floor(len(dataset) * 0.8)]
dev_set = dataset[math.floor(len(dataset) * 0.8):math.floor(len(dataset) * 0.9)]
test_set = dataset[math.floor(len(dataset) * 0.9):len(dataset) - 1]

In [8]:
vocab = set(raw_data)

In [9]:
vocab_len = len(vocab)

In [10]:
vocab_len

84

In [11]:
stoi = {char:i for i, char in enumerate(vocab)}

In [12]:
itos = {i:char for i, char in enumerate(vocab)}

In [13]:
import torch

In [14]:
import torch.nn.functional as functional

In [15]:
# make dataset
xs, ys = [], []
for data in train_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

24196

In [16]:
ws = torch.randn((vocab_len*2, vocab_len))
ws.requires_grad = True
ws.shape

torch.Size([168, 84])

In [17]:
xenc = functional.one_hot(xs[0], num_classes=vocab_len).float().flatten()
xenc @ ws

tensor([ 1.0560e+00,  1.2743e-01,  1.1645e+00,  1.2119e+00,  1.1744e+00,
         2.5078e+00,  5.8928e-01, -4.8063e-01, -2.0613e-01,  1.1768e+00,
        -2.1398e+00,  2.7658e+00,  2.5778e+00, -1.3192e+00,  1.7091e+00,
         3.9220e-01,  2.2703e-01,  1.6676e+00,  4.6383e-01, -4.0720e+00,
        -7.9494e-01, -8.5573e-01, -3.4151e+00, -1.4738e+00,  5.5158e-01,
        -2.6002e+00, -6.3340e-01, -1.0440e+00, -3.0220e-01, -2.4106e+00,
        -1.1312e-01, -1.1008e+00, -8.1029e-02,  1.1450e+00,  1.5008e+00,
         1.0374e+00, -5.2829e-01,  1.3812e-01, -1.8724e+00, -1.3136e+00,
        -5.4774e-01,  2.2659e-01,  1.6819e+00,  1.1896e-01,  2.7510e-03,
        -7.5967e-01,  7.7710e-01, -1.5769e+00, -2.3931e+00,  1.1010e+00,
        -2.9569e-01,  8.3431e-01, -3.5149e-01,  3.6013e+00,  4.6795e-01,
        -1.3597e+00, -9.3523e-01,  1.4022e+00, -3.4895e-01, -1.2198e+00,
         6.8724e-02, -6.2655e-01,  9.0110e-01,  3.1488e+00,  2.5877e-01,
         2.5003e+00,  6.7575e-01,  3.2568e-01,  5.4

In [18]:
(xenc @ ws).shape

torch.Size([84])

In [27]:
num_epochs = 100
lr = 20

In [20]:
torch.randn((3, 1))

tensor([[-0.0590],
        [-0.4435],
        [ 0.2357]])

In [28]:
for epoch in range(num_epochs):
    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr

1.588769555091858
1.5883879661560059
1.5880074501037598
1.587628960609436
1.5872516632080078
1.5868757963180542
1.5865012407302856
1.5861282348632812
1.585756778717041
1.5853866338729858
1.5850178003311157
1.5846503973007202
1.5842846632003784
1.583919882774353
1.5835565328598022
1.5831944942474365
1.5828338861465454
1.582474708557129
1.5821166038513184
1.581760048866272
1.5814045667648315
1.5810505151748657
1.580697774887085
1.5803463459014893
1.5799963474273682
1.5796475410461426
1.5793005228042603
1.5789557695388794
1.578614354133606
1.5782774686813354
1.5779497623443604
1.577636480331421
1.5773591995239258
1.5771304368972778
1.5770596265792847
1.5771197080612183
1.5779190063476562
1.5787030458450317
1.5828742980957031
1.583155870437622
1.595697045326233
1.585252046585083
1.6019314527511597
1.5835095643997192
1.5979387760162354
1.5832946300506592
1.5984103679656982
1.5824345350265503
1.59711492061615
1.5818488597869873
1.5965989828109741
1.58116614818573
1.5958120822906494
1.5805330

In [22]:
xs[:10]

tensor([[27, 27],
        [27, 57],
        [57, 26],
        [26,  8],
        [ 8, 48],
        [48, 51],
        [51,  5],
        [ 5, 59],
        [59, 40],
        [40, 52]])

In [ ]:
def forward(xs):
  xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
  logits = xenc @ ws
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  outchar = itos[torch.multinomial(probs, num_samples=1).item()]
  return outchar

def prompt(ins):
  out = ins
  for char in range(500):
    last_two = ins[-2:]
    xs = torch.tensor([[stoi[last_two[0]], stoi[last_two[1]]]])
    outs = forward(xs)
    out += outs
  return out

prompt("Rom")

'Rome(_ee//i/eee<eee)ea/eecaaeea_aeeee/,_i/_eea__:_eepe_ee//aia(/eieUeaeaef"___[e_p"e#yeaa(ee/e/e_eH/_ee"e_snk"-(_/eaeVaa(//_aeaeaaea_ee/=eieiUe/ap]ee_a/a//eeiese0ee_e/_e/(ai/aey /e(e_ee/e\\ea_eear/(i__[_ieaaeeee__(eeuee/1pa8"i/eaee(ee"_e_ea(aa_HRea_/gte/neieea_a=eepea__eeeeLeii_eepeehHe_e\\_eae/ee"[_eee_(wa_\n_e__gp_eeae_/maep_e_/a(\\eee/e__ee__xi/_ai(e_/e/_aie_eea=/e__e)e+e/ip(/eiee(eaei_i/#/eea/(ee_apeepe=_aeeea//[ep/L{e/_ii/aei/_ne_eae_/eie"N<e(aeaa(LaeiieaeeeG_aeaaeag(arsea_ae_eaeea(e{eepeiapeeeaey'

In [30]:
# make test set
xs, ys = [], []
for data in test_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

4196

In [31]:
xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
print(loss.item())

2.7738420963287354


In [32]:
# make dev set
xs, ys = [], []
for data in dev_set:
  for ch1, ch2, ch3 in zip(data, data[1:], data[2:]):
    xs.append([stoi[ch1], stoi[ch2]])
    ys.append(stoi[ch3])

xs = torch.tensor(xs)
ys = torch.tensor(ys)
num = xs.nelement()
num

3404

In [33]:
xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
logits = xenc @ ws
counts = logits.exp()
probs = counts / counts.sum(1, keepdim=True)
loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * 0.01)
print(loss.item())

1.5511993169784546


Looks like it might be overfitting? The dev set results seem fine, but the test set loss it much higher than it should be.

In [ ]:
num_epochs = 100
lr = 20
regularization = 0.01

In [ ]:
for epoch in range(num_epochs):
    xenc = functional.one_hot(xs, num_classes=vocab_len).float().flatten(start_dim=1)
    logits = xenc @ ws
    counts = logits.exp()
    probs = counts / counts.sum(1, keepdim=True)
    loss = -probs[torch.arange(int(num / 2)), ys].log().mean() + (ws.mean() * regularization)
    print(loss.item())

    ws.grad = None
    loss.backward()
    ws.data += ws.grad * -lr